Primero vamos a conectarnos con el servidor de origen y listar sus proyectos

In [1]:
from redminelib import Redmine
import cfg_src_rm

if cfg_src_rm.ignore_cert:
    src_rm = Redmine(cfg_src_rm.server_url,key=cfg_src_rm.api_key, requests={'verify': False})
else:
    src_rm = Redmine(cfg_src_rm.server_url,key=cfg_src_rm.api_key)

src_prjs = src_rm.project.all()

print("Proyectos del servidor origen:")
for p in src_prjs:
    print ("\t",p.identifier," \t| ",p.name)


Proyectos del servidor origen:
	 est  	|  EST
	 gris-gregor  	|  GRIS@GREGOR
	 mcao-aiv4  	|  MCAO-AIV4
	 loaddsm-tests  	|  MICAL-GRIS
	 solarnet-ifu  	|  Solarnet IFU
	 mcao-aiv  	|  MCAO-AIV
	 mcao-aiv2  	|  MCAO-AIV2
	 mcao-aiv3  	|  MCAO-AIV3
	 mical-gris-test2  	|  MICAL-GRIS test2
	 mical-gris  	|  MICAL-GRISant
	 rees  	|  Reespirator 2020
	 solarnet-ifu2  	|  Solarnet IFU2


Ahora vamos a averiguar los identificadores de los campos personalizados que necesitaremos, mientras construimos un diccionario de ellos:

In [2]:
src_cfs = {}
for cf in src_rm.custom_field.all():
    print(cf)
    src_cfs[cf.name] = cf
    
print(src_cfs.keys())

Supervisor
IssChapter
IssTitle
IssHierarchyDiagram
IssDependenceDiagram
IssHierarchyDiagram
IssDependenceDiagram
IssPrefix
IssCounter
Report
Start date
Working days
Official name
Collaborators
WrkloadPct
Depend
DependII
dict_keys(['Supervisor', 'IssChapter', 'IssTitle', 'IssHierarchyDiagram', 'IssDependenceDiagram', 'IssPrefix', 'IssCounter', 'Report', 'Start date', 'Working days', 'Official name', 'Collaborators', 'WrkloadPct', 'Depend', 'DependII'])


Listaremos las tareas mientras construimos un diccionario para saber encontrar las tareas por subject

In [3]:
src_prj = src_rm.project.get(cfg_src_rm.project_id)
print ("Obtenemos proyecto: ",src_prj.identifier," | ",src_prj.name)

src_prj_issues = src_rm.issue.filter(
    project_id=src_prj.id,
    sort='subject:asc'
)

src_tsks = {}
for i in src_prj_issues:
    print(i.subject)
    src_tsks[i.subject] = src_rm.issue.get(i.id)
    
print(src_tsks.keys())

Obtenemos proyecto:  solarnet-ifu  |  Solarnet IFU
IFU-0001
IFU-0002
IFU-0003
IFU-0004
IFU-0005
IFU-0006
IFU-0007
IFU-0008
IFU-0009
IFU-0010
IFU-0011
IFU-0012
IFU-0013
IFU-0014
IFU-0015
IFU-0018
IFU-0019
IFU-0020
IFU-0021
IFU-0022
IFU-0023
IFU-0024
IFU-0025
IFU-0026
IFU-0027
IFU-0028
IFU-0029
IFU-0030
IFU-0031
IFU-0032
IFU-0033
IFU-0034
dict_keys(['IFU-0001', 'IFU-0002', 'IFU-0003', 'IFU-0004', 'IFU-0005', 'IFU-0006', 'IFU-0007', 'IFU-0008', 'IFU-0009', 'IFU-0010', 'IFU-0011', 'IFU-0012', 'IFU-0013', 'IFU-0014', 'IFU-0015', 'IFU-0018', 'IFU-0019', 'IFU-0020', 'IFU-0021', 'IFU-0022', 'IFU-0023', 'IFU-0024', 'IFU-0025', 'IFU-0026', 'IFU-0027', 'IFU-0028', 'IFU-0029', 'IFU-0030', 'IFU-0031', 'IFU-0032', 'IFU-0033', 'IFU-0034'])


Ahora vamos a conectarnos con el servidor de destino y listar sus proyectos

In [4]:
from redminelib import Redmine
import cfg_dst_rm

if cfg_dst_rm.ignore_cert:
    dst_rm = Redmine(cfg_dst_rm.server_url,key=cfg_dst_rm.api_key, requests={'verify': False})
else:
    dst_rm = Redmine(cfg_dst_rm.server_url,key=cfg_dst_rm.api_key)

dst_prjs = dst_rm.project.all()

print("Proyectos del servidor destino:")
for p in dst_prjs:
    print ("\t",p.identifier," \t| ",p.name)

Proyectos del servidor destino:
	 est  	|  EST
	 gris-gregor  	|  GRIS@GREGOR
	 mcao-aiv4  	|  MCAO-AIV4
	 loaddsm-tests  	|  MICAL-GRIS
	 solarnet-ifu  	|  Solarnet IFU
	 mcao-aiv  	|  MCAO-AIV
	 mcao-aiv2  	|  MCAO-AIV2
	 mcao-aiv3  	|  MCAO-AIV3
	 mical-gris-test2  	|  MICAL-GRIS test2
	 mical-gris  	|  MICAL-GRISant
	 rees  	|  Reespirator 2020
	 solarnet-ifu2  	|  Solarnet IFU2


Ahora vamos a averiguar los identificadores de los campos personalizados que necesitaremos, mientras construimos un diccionario de ellos:

In [5]:
dst_cfs = {}
for cf in dst_rm.custom_field.all():
    print(cf)
    dst_cfs[cf.name] = cf
    
print(dst_cfs.keys())

Supervisor
IssChapter
IssTitle
IssHierarchyDiagram
IssDependenceDiagram
IssHierarchyDiagram
IssDependenceDiagram
IssPrefix
IssCounter
Report
Start date
Working days
Official name
Collaborators
WrkloadPct
Depend
DependII
dict_keys(['Supervisor', 'IssChapter', 'IssTitle', 'IssHierarchyDiagram', 'IssDependenceDiagram', 'IssPrefix', 'IssCounter', 'Report', 'Start date', 'Working days', 'Official name', 'Collaborators', 'WrkloadPct', 'Depend', 'DependII'])


Listaremos las tareas mientras construimos un diccionario para saber encontrar las tareas por subject

In [6]:
dst_prj = dst_rm.project.get(cfg_dst_rm.project_id)
print ("Obtenemos proyecto: ",dst_prj.identifier," | ",dst_prj.name)

dst_prj_issues = dst_rm.issue.filter(
    project_id=dst_prj.id,
    sort='subject:asc'
)

dst_tsks = {}
for i in dst_prj_issues:
    print(i.subject)
    dst_tsks[i.subject] = dst_rm.issue.get(i.id)
    
print(dst_tsks.keys())

Obtenemos proyecto:  solarnet-ifu2  |  Solarnet IFU2
IFU-0001
IFU-0002
IFU-0003
IFU-0004
IFU-0005
IFU-0006
IFU-0007
IFU-0008
IFU-0009
IFU-0010
IFU-0011
IFU-0012
IFU-0013
IFU-0014
dict_keys(['IFU-0001', 'IFU-0002', 'IFU-0003', 'IFU-0004', 'IFU-0005', 'IFU-0006', 'IFU-0007', 'IFU-0008', 'IFU-0009', 'IFU-0010', 'IFU-0011', 'IFU-0012', 'IFU-0013', 'IFU-0014'])


Definiremos cómo se copian las tareas, según exista ya la tarea destino, o según sea una tarea a crear

In [7]:
def calculamos_cfs(srcTsk):
    tmpCfs=[]
    for k in src_cfs.keys():
        print("\n******",k)
        print("cf:",src_cfs[k].name)
        if k not in dst_cfs.keys():
            print("WARNING: no existe en el servidor destino el campo",k)
            
        else:
            print("cfsid:",src_cfs[k].id)
            cv = srcTsk.custom_fields.get(src_cfs[k].id)
            if cv is not None:
                if hasattr(cv,'value'):
                    cv_val = cv.value
                    print("tsk.cv",cv_val)
                    tmpCfs += [{'id' : src_cfs[k].id, 'value': cv_val}]
                    
    print("Result:",tmpCfs)
    return tmpCfs



def update_tsk(srcTsk,dstTsk):
    global dst_rm,dst_cfs,src_cfs
    print("\n>>>>>",srcTsk)
    tmpCfs=calculamos_cfs(srcTsk)
    dst_rm.issue.update(
        dstTsk.id,
        tracker_id=srcTsk.tracker.id,
        description=srcTsk.description+"_pepe",
        #status_id=srcTsk.status_id,
        #priority_id=srcTsk.priority_id,
        #assigned_to_id=srcTsk.assigned_to_id,
        #parent_issue_id=srcTsk.parent_issue_id,
        start_date=srcTsk.start_date,
        due_date=srcTsk.due_date,
        #estimated_hours=srcTsk.estimated_hours,
        custom_fields=tmpCfs,
        done_ratio=srcTsk.done_ratio,
    )
    
def copy_tsk(srcTsk):
    global dst_rm,dst_cfs,src_cfs
    print("\n>>>>>",srcTsk)
    tmpCfs=calculamos_cfs(srcTsk)
    ret = dst_rm.issue.create(
        project_id=dst_prj.id,
        tracker_id=srcTsk.tracker.id,
        description=srcTsk.description,
        #status_id=srcTsk.status_id,
        #priority_id=srcTsk.priority_id,
        #assigned_to_id=srcTsk.assigned_to_id,
        #parent_issue_id=srcTsk.parent_issue_id,
        start_date=srcTsk.start_date,
        due_date=srcTsk.due_date,
        #estimated_hours=srcTsk.estimated_hours,
        custom_fields=tmpCfs,
        done_ratio=srcTsk.done_ratio
    )
    return ret

Ahora identificaremos, para cada tarea de origen, su tarea destino

In [8]:
for k in src_tsks.keys():
    if k in dst_tsks.keys():
        print("ya existe")
        update_tsk(src_tsks[k],dst_tsks[k])
        
    else:
        print("hará falta una tarea nueva")
        dst_tsks[k] = copy_tsk(src_tsks[k])
        
    src_tsks[k]['dst'] = dst_tsks[k]

ya existe

>>>>> IFU-0001

****** Supervisor
cf: Supervisor
cfsid: 14

****** IssChapter
cf: IssChapter
cfsid: 15
tsk.cv IFU-z.1.

****** IssTitle
cf: IssTitle
cfsid: 16
tsk.cv Aprendizaje y traspaso información nueva IngOpt

****** IssHierarchyDiagram
cf: IssHierarchyDiagram
cfsid: 19

****** IssDependenceDiagram
cf: IssDependenceDiagram
cfsid: 20

****** IssPrefix
cf: IssPrefix
cfsid: 21

****** IssCounter
cf: IssCounter
cfsid: 23

****** Report
cf: Report
cfsid: 25

****** Start date
cf: Start date
cfsid: 26

****** Working days
cf: Working days
cfsid: 27

****** Official name
cf: Official name
cfsid: 28

****** Collaborators
cf: Collaborators
cfsid: 29
tsk.cv ['34']

****** WrkloadPct
cf: WrkloadPct
cfsid: 31
tsk.cv 20

****** Depend
cf: Depend
cfsid: 32
tsk.cv 

****** DependII
cf: DependII
cfsid: 33
tsk.cv 
Result: [{'id': 15, 'value': 'IFU-z.1.'}, {'id': 16, 'value': 'Aprendizaje y traspaso información nueva IngOpt'}, {'id': 29, 'value': ['34']}, {'id': 31, 'value': '20'}, {'id'

Ahora trataremos la copia de cada tarea, dependiendo de si es una actualización o es una creación

In [9]:
print("hecho")

hecho
